In [27]:
import pandas as pd
import numpy as np
pd.set_option('display.colheader_justify', 'left')  # Left-align column headers
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pytz
from tqdm import tqdm  # Visualize loop progress
# from sklearn.linear_model import LinearRegression
from tenacity import retry, stop_after_attempt, wait_fixed
import openpyxl
import os
import requests
import yfinance as yf

import matplotlib.dates as mdates
from matplotlib.ticker import MultipleLocator

In [28]:
# tradingview lightweight-charts
from lightweight_charts import Chart
import asyncio
import nest_asyncio
# Apply nest_asyncio AT START to allow nested event loops (required for Jupyter)
nest_asyncio.apply()
#
from dotenv import load_dotenv

In [29]:
# load env vars
load_dotenv()

True

In [30]:
# #####---------------------#####s
# screen_date = '2025-03-07'
screen_date = '2025-03-13'
# #####---------------------#####
# read pre market data from TradingView data via csv
trading_view_df = pd.read_csv(f"/Users/sudz4/Desktop/MATT-MATTHEW/matt-matthew/tv_screen_gap-up_{screen_date}.csv")

In [31]:
# print(trading_view_df.columns)

In [32]:
# CREATE CATEGORIES FOR MARKET CAP
def categorize_market_cap(df):
    # categorize stocks groups by market cap
    df['Market capitalization'] = pd.to_numeric(df['Market capitalization'], errors='coerce')
    conditions = [
        (df['Market capitalization'] >= 200_000_000_000),  # Titans
        (df['Market capitalization'] >= 10_000_000_000) & (df['Market capitalization'] < 200_000_000_000),  # Large caps
        (df['Market capitalization'] >= 2_000_000_000) & (df['Market capitalization'] < 10_000_000_000),  # Mid caps
        (df['Market capitalization'] >= 300_000_000) & (df['Market capitalization'] < 2_000_000_000),  # Small caps
        (df['Market capitalization'] > 50_000_000) & (df['Market capitalization'] < 300_000_000),  # Micro caps
        (df['Market capitalization'] <= 50_000_000)  # Shrimp
    ]
    # marekt cap categories list
    categories = ['Titans', 'Large caps', 'Mid caps', 'Small caps', 'Micro caps', 'Shrimp']
    df['marketCapType'] = np.select(conditions, categories, default='Undefined')
    return df
# execute categorization
category_setup_df = categorize_market_cap(trading_view_df).copy()
# drop Undefined marketCapType
category_setup_df = category_setup_df[category_setup_df['marketCapType'] != 'Undefined']

# convert necessary columns to numeric
def convert_columns_to_numeric(df, columns):
    """Convert specified columns to numeric types."""
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

# list of columns to convert
numeric_columns = [
    'Market capitalization', 'Float shares outstanding', 'Relative Volume 1 day',
    'Relative Volume at Time', 'Pre-market Change %', 'Pre-market Gap %',
    'Price', 'Volume Weighted Average Price 1 day', 'Volatility 1 day',
    'Volatility 1 week', 'Volatility 1 month', 'Pre-market Volume',
    'Exponential Moving Average (3) 1 day', 'Exponential Moving Average (7) 1 day'
]

# apply conversion
category_setup_df = convert_columns_to_numeric(category_setup_df, numeric_columns)

# CRITERIA CONFIGURATION FOR EACH MARKET CAP CATEGORY
criteria_config = {
    "Titans": {
        "pre_market_change_pct_threshold": 0.002,  # 0.2% for Titans
        "float_shares_outstanding_threshold": 1_000_000_000,  # 1 billion shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.03,
        "pre_market_gap_percentage_threshold": 0.001,  # 0.1%
        "pre_market_vwap_drawdown_threshold": 0.003,  # 0.3% drawdown from VWAP
        "pre_market_volume_threshold": 50_000  # Minimum pre-market volume
    },
    "Large caps": {
        "pre_market_change_pct_threshold": 0.005,  # 0.5% for Large caps
        "float_shares_outstanding_threshold": 200000000,  # 200 million shares
        "relative_volume_threshold": 1.3,  # More inclusive
        "relative_volume_at_time_threshold": 0.04,  # More inclusive
        "pre_market_gap_percentage_threshold": 0.005,  # 0.5%
        "pre_market_vwap_drawdown_threshold": 0.004,  # 0.4% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Midlers": { 
        "pre_market_change_pct_threshold": 0.02,  # 2% for Midlers 
        "float_shares_outstanding_threshold": 50000000,  # 50 million shares
        "relative_volume_threshold": 1.3,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.02,
        "pre_market_vwap_drawdown_threshold": 0.005,  # 0.5% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Small caps": {
        "pre_market_change_pct_threshold": 0.03,  # 3% for Small caps
        "float_shares_outstanding_threshold": 20000000,  # 20 million shares
        "relative_volume_threshold": 1.2,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.03,
        "pre_market_vwap_drawdown_threshold": 0.006,  # 0.6% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Micro caps": {
        "pre_market_change_pct_threshold": 0.04,  # 4% for Micro caps
        "float_shares_outstanding_threshold": 5000000,  # 5 million shares
        "relative_volume_threshold": 1.1,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.04,
        "pre_market_vwap_drawdown_threshold": 0.007,  # 0.7% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    },
    "Shrimp": {
        "pre_market_change_pct_threshold": 0.05,  # 5% for Shrimp
        "float_shares_outstanding_threshold": 1000000,  # 1 million shares
        "relative_volume_threshold": 1.0,
        "relative_volume_at_time_threshold": 0.05,
        "pre_market_gap_percentage_threshold": 0.05,
        "pre_market_vwap_drawdown_threshold": 0.008, # 0.8% drawdown from VWAP
        "pre_market_volume_threshold": 50000  # Minimum pre-market volume
    }
}

# FILTER STOCKS BASED ON CONFIGURATION CRITERIA - STAGING
def filter_stocks(df, config):
    # stock filtering conditions >=< based on criteria
    conditions = (
        (df['Pre-market Change %'] >= config.get('pre_market_change_pct_threshold', 0)) &
        (df['Float shares outstanding'] <= config.get('float_shares_outstanding_threshold', float('inf'))) &
        (df['Relative Volume 1 day'] >= config.get('relative_volume_threshold', 0)) &
        (df['Relative Volume at Time'] >= config.get('relative_volume_at_time_threshold', 0)) &
        (df['Pre-market Gap %'] >= config.get('pre_market_gap_percentage_threshold', 0)) &
        (df['Price'] >= df['Volume Weighted Average Price 1 day'] * (1 - config.get('pre_market_vwap_drawdown_threshold', 0))) &
        (df['Volatility 1 day'] >= df['Volatility 1 week']) &
        (df['Volatility 1 day'] >= df['Volatility 1 month']) &
        (df['Pre-market Volume'] >= config.get('pre_market_volume_threshold', 0))
    )
    return df[conditions]

# SCREEN STOCKS BY CATEGORY
def screen_stocks_by_category(df, category):
    """Filter stocks in a category using predefined criteria."""
    config = criteria_config.get(category, {})
    filtered_df = filter_stocks(df, config)
    return filtered_df

# EXECUTE KEY SCREENING FUNCTION AND CREATE DATAFRAME
smash_df = pd.DataFrame()
categories = category_setup_df['marketCapType'].unique()

for category in categories:
    category_df = category_setup_df[category_setup_df['marketCapType'] == category]
    gap_up_stage_df = screen_stocks_by_category(category_df, category)
    smash_df = pd.concat([smash_df, gap_up_stage_df], ignore_index=True)

# rendered column list and ordering
cols_list = [
    'Symbol', 
    'Description', 
    'marketCapType', 
    'Pre-market Change %', 
    'Pre-market Gap %', 
    'Market capitalization',
    'Price', 
    'Pre-market Open', 
    'Industry', 
    'Index', 
    'Sector', 
    'Exchange',
    'Recent earnings date', 
    'Upcoming earnings date', 
    'Float shares outstanding', 
    'Average Volume 10 days',
    'Average Volume 30 days', 
    'Average Volume 90 days',
    'Relative Volume 1 day', 
    'Relative Volume 5 minutes', 
    'Relative Volume 30 minutes', 
    'Relative Volume at Time', 
    'Analyst Rating',
    'Technical Rating 5 minutes',
    'Exponential Moving Average (3) 1 day',
    'Exponential Moving Average (7) 1 day'
]

# filter columns to only include those present in the DataFrame
existing_cols = [col for col in cols_list if col in smash_df.columns]
smash_df = smash_df[existing_cols]

# sort and reset index
smash_df = smash_df.sort_values(
    by=['Pre-market Change %', 'Price'],
    ascending=[False, False]).reset_index(drop=True)

print(f"{smash_df.shape[0]} stocks found.")
display(smash_df.head(20))
# display(smash_df)
# # ALL SMASH
# display(smash_df)

5 stocks found.


,Symbol,Description,marketCapType,Pre-market Change %,Pre-market Gap %,Market capitalization,Price,Pre-market Open,Industry,Index,Sector,Exchange,Recent earnings date,Upcoming earnings date,Float shares outstanding,Average Volume 10 days,Average Volume 30 days,Average Volume 90 days,Relative Volume 1 day,Relative Volume 5 minutes,Relative Volume 30 minutes,Relative Volume at Time,Analyst Rating,Technical Rating 5 minutes,Exponential Moving Average (3) 1 day,Exponential Moving Average (7) 1 day
0,QBTS,D-Wave Quantum Inc.,Mid caps,4.639175,5.154639,2.072168e+09,6.6773,6.12,Computer processing hardware,"Russell 2000, Russell 3000, Mini-Russell 2000",Electronic technology,NYSE,2025-03-13,2025-05-14,2.579080e+08,46673777.6,5.843437e+07,6.726860e+07,1.393116,0.202928,0.238212,3.466960,Strong buy,Buy,6.084330,5.724765
1,MRCY,Mercury Systems Inc,Mid caps,3.301663,0.712589,2.645367e+09,44.3250,42.40,Aerospace & defense,"NASDAQ Composite, Russell 2000, Russell 3000, NASDAQ Industrials, Mini-Russell 2000",Electronic technology,NASDAQ,2025-02-04,2025-05-06,5.829666e+07,449847.6,5.846902e+05,5.200861e+05,0.299273,0.315055,0.180068,2.752139,Neutral,Buy,43.488355,43.378683
2,OII,"Oceaneering International, Inc.",Mid caps,1.549587,0.878099,2.000134e+09,19.8700,19.53,Industrial machinery,"Russell 2000, Nasdaq US Small Cap Growth, Russell 3000, Mini-Russell 2000, PHLX Oil Service Sector",Producer manufacturing,NYSE,2025-02-19,2025-04-23,9.858265e+07,1147332.4,9.326179e+05,8.227597e+05,0.310097,0.311849,0.079508,2.268132,Neutral,Neutral,19.606710,19.806086
3,ALGM,"Allegro MicroSystems, Inc.",Mid caps,1.042502,1.042502,4.835123e+09,26.2600,25.20,Semiconductors,"NASDAQ Composite, S&P MidCap 400, Nasdaq US Small Cap Growth, Russell 3000, Russell 1000, NASDAQ Computer, NASDAQ Clean Edge Green Energy",Electronic technology,NASDAQ,2025-01-30,2025-05-07,1.222570e+08,5526110.5,3.836668e+06,2.668664e+06,0.270426,0.213505,0.099449,0.944560,Strong buy,Neutral,25.846347,25.817363
4,HCM,HUTCHMED (China) Limited,Mid caps,0.710136,1.355713,2.719611e+09,15.4400,15.70,Pharmaceuticals: major,"NASDAQ Composite, NASDAQ Biotechnology",Health technology,NASDAQ,2024-07-31,2025-03-19,1.703187e+08,71942.0,7.818113e+04,7.740687e+04,0.109959,0.762165,0.265550,0.251788,Strong buy,Neutral,15.540618,15.732745


In [33]:
# group and count by sector to new df called sector_count_df
sector_count_df = smash_df.groupby('Sector').size().reset_index(name='Count').sort_values(by='Count', ascending=False)
# print the date and time of the screen
print(f"Screening date: {screen_date}")
print(f"Total stocks returned: {len(smash_df)}")
display(sector_count_df)

Screening date: 2025-03-13
Total stocks returned: 5


,Sector,Count
0,Electronic technology,3
1,Health technology,1
2,Producer manufacturing,1


In [34]:
smash_list = smash_df['Symbol'].tolist()
print(len(smash_list))
print(smash_list)

5
['QBTS', 'MRCY', 'OII', 'ALGM', 'HCM']


In [35]:
# Create a filter for EMA3 > EMA7
smash_ema_df = smash_df['Exponential Moving Average (3) 1 day'] > smash_df['Exponential Moving Average (7) 1 day']

# Apply the filter to smash_df
filtered_smash_df = smash_df[smash_ema_df]

# Print the number of stocks that meet the criteria
print(f"Number of stocks where EMA3 > EMA7: {len(filtered_smash_df)}")
print("\nFiltered stocks:")
print(filtered_smash_df['Symbol'].tolist())

Number of stocks where EMA3 > EMA7: 3

Filtered stocks:
['QBTS', 'MRCY', 'ALGM']
